#Installing required dependencies

In [1]:
!pip install sentence-splitter

     |████████████████████████████████| 44 kB 1.6 MB/s 


In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 53.9 MB/s 
     |████████████████████████████████| 6.6 MB 49.4 MB/s 
     |████████████████████████████████| 596 kB 61.3 MB/s 
     |████████████████████████████████| 77 kB 7.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
! pip install SentencePiece

     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [4]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from sentence_splitter import SentenceSplitter, split_text_into_sentences

#Download model using Huggingface 

In [5]:
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

#Processing for a single sentence

In [6]:
sentence="What a great day it is today. I am loving it."

In [7]:
get_response(sentence, 5)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3516: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


['Today is a great day.',
 'It is a great day and I am enjoying it.',
 'It is a great day.',
 'Today is a great day and I am loving it.',
 'Today is a great day and I am enjoying it.']

#Processing for an entire paragraph

In [8]:
paragraph = "The global efforts to control COVID-19 are threatened by the rapid emergence of novel SARS-CoV-2 variants that may display undesirable characteristics such as immune escape or increased pathogenicity. Early prediction of emerging strains could be vital to pandemic preparedness but remains an open challenge. "
print(paragraph)

The global efforts to control COVID-19 are threatened by the rapid emergence of novel SARS-CoV-2 variants that may display undesirable characteristics such as immune escape or increased pathogenicity. Early prediction of emerging strains could be vital to pandemic preparedness but remains an open challenge. 


In [9]:
splitter = SentenceSplitter(language='en') #splits the paragraph into a list of sentences

In [10]:
sentence_list = splitter.split(paragraph)
sentence_list

['The global efforts to control COVID-19 are threatened by the rapid emergence of novel SARS-CoV-2 variants that may display undesirable characteristics such as immune escape or increased pathogenicity.',
 'Early prediction of emerging strains could be vital to pandemic preparedness but remains an open challenge.']

#Making paraphrases of each sentence in the list of sentences

In [11]:
paraphrase = []

for i in sentence_list: 
  a = get_response(i,1)
  paraphrase.append(a)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3516: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [12]:
paraphrase

[['The global efforts to control COVID-19 are threatened by the rapid emergence of novel SARS-CoV-2 variant that may display undesirable characteristics.'],
 ['Predicting emerging strains is an open challenge.']]

In [16]:
paraphrase_list = [' '.join(x) for x in paraphrase]
paraphrase_list

['The global efforts to control COVID-19 are threatened by the rapid emergence of novel SARS-CoV-2 variant that may display undesirable characteristics.',
 'Predicting emerging strains is an open challenge.']

In [17]:
# Combines the above list into a paragraph
paraphrase_final = [' '.join(x for x in paraphrase_list) ]
paraphrased_text = str(paraphrase_list).strip('[]').strip("'")
paraphrased_text

"The global efforts to control COVID-19 are threatened by the rapid emergence of novel SARS-CoV-2 variant that may display undesirable characteristics.', 'Predicting emerging strains is an open challenge."

In [18]:
# Comparison of the original (context variable) and the paraphrased version (paraphrase3 variable)

print(paragraph)
print(paraphrased_text)

The global efforts to control COVID-19 are threatened by the rapid emergence of novel SARS-CoV-2 variants that may display undesirable characteristics such as immune escape or increased pathogenicity. Early prediction of emerging strains could be vital to pandemic preparedness but remains an open challenge. 
The global efforts to control COVID-19 are threatened by the rapid emergence of novel SARS-CoV-2 variant that may display undesirable characteristics.', 'Predicting emerging strains is an open challenge.
